In [1]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/amjalled/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [2]:
using DrWatson
@quickactivate :Catalyst

┌ Info: Precompiling Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
└ @ Base loading.jl:1260


In [3]:
using Distributed
N = 10
addprocs(N)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [4]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst

      From worker 3:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 8:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 7:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 10:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 9:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 5:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 6:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 11:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 4:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 2:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Pr

In [5]:
function calibrate(cma,N)
    Init = [0.013235189689493238, 1., 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0, 0], [1, 50, 50, 30, 30]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    input_exp = []
    output_exp = []
    for row in CSV.File(datadir("experiment/SBA15_interp_12.csv"); delim = ",")
        push!(input_exp, row.I)
        push!(output_exp, row.O)
    end
    n = 2
    nels = (5n, convert(Int,2.5n),n)
    S = Vec((0.0,0.0,0.0))
    E = Vec((1.0,0.5,0.2))
    grid = generate_grid(Tetrahedron, nels, S, E)
    faces = union(getfaceset(grid,"top"),getfaceset(grid,"bottom"),getfaceset(grid,"left"),getfaceset(grid,"right"),
                   getfaceset(grid,"front"),getfaceset(grid,"back"))
    addfaceset!(grid, "1", faces)
    
    while isempty(es.stop()) 
        solutions = es.ask() 
        fitness = pmap(function eval(input_args) 
                            try 
                               return Catalyst.solve(input_args[1], input_args[2], input_args[3], 
                                          input_exp, output_exp, progress=false, 
                                          microcomp_type=:nonlinear, 
                                          Q=input_args[4], kₙ=input_args[5], 
                                          calibration=true,w=(2.2954e-4*(1/0.37)),micromesh=grid)
                             catch err #in case we loose newton convergence in a microscopic computation 
                                return 1e10
                             end
                        end
                            ,solutions) 
        es.tell(solutions,fitness) 
        es.logger.add() 
        es.disp() 
    end
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 5 (seed=277535, Wed Aug 12 13:04:51 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 2.654572910362567e+02 1.0e+00 4.89e-01  4e-01  5e-01 15:16.2
    2     20 2.571845310428248e+02 1.2e+00 5.40e-01  4e-01  6e-01 28:54.4
    3     30 2.358667220357502e+02 1.7e+00 6.83e-01  6e-01  8e-01 42:32.4
    4     40 2.013051778287392e+02 1.7e+00 8.90e-01  7e-01  1e+00 56:21.6
    5     50 1.875166873470328e+02 1.9e+00 9.69e-01  8e-01  1e+00 70:14.6
    6     60 1.718234857956428e+02 1.9e+00 1.04e+00  8e-01  1e+00 83:53.6
    7     70 1.554590401192146e+02 2.0e+00 1.04e+00  8e-01  1e+00 97:36.1
    8     80 1.699243484716739e+02 2.2e+00 1.11e+00  9e-01  1e+00 112:16.8
    9     90 1.429455985288713e+02 2.4e+00 1.27e+00  1e+00  2e+00 126:29.8
   10    100 3.394462730756821e+01 2.5e+00 1.43e+00  1e+00  2e+00 140:43.7
   11    110 4.760707918765993e+00 2.5e+00 1.39e+00  1e+00  2e+00 154:18.0
   12    120 1.5175